In [ ]:
# from google.colab import drive
# drive.mount('drive')
# !sudo pip install pycountry_convert
# !sudo pip install imdbpy
# !sudo pip install iso3166
import requests
import numpy as np
import pandas as pd
import re
import json
from iso3166 import countries
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from imdb import IMDb
import csv

def map_country(name):
  if name == 'UK':
    return 'United Kingdom'
  elif name == 'USA':
    return 'United States'
  else:
    return name

def get_company(name,year):
  ia = IMDb()
  s = ia.search_movie(str(name))
  b = []
  if year.isdigit():
    b = [i for i in range(len(s)) if 'year' in s[i] and s[i]['year']==int(year)]
  companies = []

  if b:

    dp = s[b[0]]
    ia.update(dp)
    g = np.array(dp.get('production companies'))

    if g.size:
      for i in range(g.size):
        companies += [g[i]['name']]
      return '|'.join(companies)
  else:
    return ''



def convert_continent(countriess):

 continents = {
     'NA': 'North America',
     'SA': 'South America',
     'AS': 'Asia',
     'OC': 'Australia',
     'AF': 'Africa',
     'EU': 'Europe'
 }

 final = ""
 if countriess:
  countriess = countriess.split("|")
  countriess = [i.strip() for i in countriess]

  if countriess:
    result = [continents[country_alpha2_to_continent_code(country_name_to_country_alpha2(map_country(country)))] for country in countriess]
    final =  "|".join(result)
 return final


data =pd.read_csv("drive/MyDrive/movies.dat",sep="::",header=None)
data = np.array(data)
row = 0

with open('dataset.csv', mode='w') as data_file:
  for j in range(1000):
    g = []
    counter_minus = 1
    sample = data[j,1]
    splitted= str.split(sample)
    split_character = list(splitted[-2])

    if split_character[0]=='(' and split_character[-1]==')':
      counter_minus=2

    year = re.search('\(([^)]+)', splitted[-1]).group(1)
    query = [splitted[0]]

    for i in range(1,len(splitted)-counter_minus):
      query +=['+'+splitted[i]]

    if query[-1] in ['+The','+Le','+La','+Les','+A','+An','+Il','+Lo',"+L'","+Der","+Das","+Los","+Las","+El","+La","+Unas","+Un","+Una","+Unos"]:
      query[-1] = query[-1].replace("+", "")
      query[0] = "+" + query[0]
      query = query[-1:]+query[:-1]

    query = ''.join(query)
    response = "http://www.omdbapi.com/?t="+query+"+&y="+year+"&apikey=6f94dfca"
  # print("http://www.omdbapi.com/?t=Milk+Money+&y=1994&apikey=6f94dfca")
    print(response)
    response = requests.get(response)
    a= response.json()
    print(a)
    if a['Response']=='True':
      g += [j+1,a['Title'],a['Year'],a['Genre'].replace(',', '|'),a['Language'].replace(',', '|'),a['Country'].replace(',', '|'),convert_continent(a['Country'].replace(',', '|')),get_company(a['Title'],a['Year']),a['Runtime'].replace('min', ''),a['imdbRating'],a['imdbID']]
    else:
      g += [0]*11


    data_writer = csv.writer(data_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    data_writer.writerow(g)

# g = np.array(g)
# df = pd.DataFrame (g)
# filepath = 'dataset6.xlsx'
# df.to_excel(filepath, index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


http://www.omdbapi.com/?t=Toy+Story+&y=1995&apikey=6f94dfca
{'Title': 'Toy Story', 'Year': '1995', 'Rated': 'G', 'Released': '22 Nov 1995', 'Runtime': '81 min', 'Genre': 'Animation, Adventure, Comedy', 'Director': 'John Lasseter', 'Writer': 'John Lasseter, Pete Docter, Andrew Stanton', 'Actors': 'Tom Hanks, Tim Allen, Don Rickles', 'Plot': "A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.", 'Language': 'English', 'Country': 'United States', 'Awards': 'Won 1 Oscar. 27 wins & 23 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.3/10'}, {'Source': 'Rotten Tomatoes', 'Value': '100%'}, {'Source': 'Metacritic', 'Value': '95/100'}], 'Metascore': '95', 'imdbRating': '8.3', 'imdbVotes': '938,867', 'imdbID': 'tt0114709', 'Type': 'movie', 'DVD': '23 Mar 

IndexError: ignored